In [ ]:
#%% 
#Imports

import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.datasets import fetch_openml

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.metrics import (
    adjusted_rand_score,
    normalized_mutual_info_score,
    silhouette_score,
)

rng = np.random.RandomState(42)

In [ ]:
#%% 
#Load the MNIST data

X_mnist, y_mnist = fetch_openml(
    name="mnist_784",
    return_X_y=True,
    as_frame=False,
)

# Ensure labels are integers 0–9
y_mnist = y_mnist.astype(int)

print("X_mnist shape:", X_mnist.shape)
print("y_mnist shape:", y_mnist.shape)
print("Unique labels:", np.unique(y_mnist))

In [ ]:
#%% 
#Visualise sample images: 1 row per digit and 5 images per digit

n_classes = 10
samples_per_class = 5
img_size = 28

fig, axes = plt.subplots(
    n_classes,
    samples_per_class,
    figsize=(samples_per_class * 2, n_classes * 2),
)

for digit in range(n_classes):
    idx = np.where(y_mnist == digit)[0]
    chosen = rng.choice(idx, size=samples_per_class, replace=False)

    for j, i in enumerate(chosen):
        ax = axes[digit, j]
        ax.imshow(X_mnist[i].reshape(img_size, img_size), cmap="gray")
        ax.axis("off")
        if j == 0:
            ax.set_ylabel(str(digit), rotation=0, labelpad=20, fontsize=10)

plt.suptitle("Sample MNIST images: rows = digits 0–9, columns = examples", y=1)
plt.tight_layout()
plt.show()


In [ ]:
#%% 
# PCA for dimensionality reduction

# Choose a reasonable number of PCA components
n_components_pca = 50

pca = PCA(n_components=n_components_pca, random_state=42)
X_pca = pca.fit_transform(X_mnist)

print("Original dimensionality:", X_mnist.shape[1])
print("Reduced dimensionality:", X_pca.shape[1])


In [ ]:
#%% 
# Helper function to compute clustering metrics:(ARI)(NMI) and Silhouette Score
def evaluate_clustering(X, y_true, labels, sample_size=5000, random_state=42):
    ari = adjusted_rand_score(y_true, labels)
    nmi = normalized_mutual_info_score(y_true, labels)

    n_samples = min(sample_size, X.shape[0])
    idx = rng.choice(X.shape[0], size=n_samples, replace=False)
    sil = silhouette_score(X[idx], labels[idx])

    return ari, nmi, sil


In [ ]:
#%% 
# K-Means experiments with different hyperparameters: n_clusters & n_init

kmeans_results = []
best_kmeans_model = None
best_kmeans_config = None
best_kmeans_score = -np.inf

n_clusters_list = [8, 10, 12]
n_init_list = [10, 20]

for n_clusters in n_clusters_list:
    for n_init in n_init_list:
        print(f"Training KMeans: n_clusters={n_clusters}, n_init={n_init} ...")
        kmeans = KMeans(
            n_clusters=n_clusters,
            n_init=n_init,
            random_state=42,
        )
        labels = kmeans.fit_predict(X_pca)

        ari, nmi, sil = evaluate_clustering(X_pca, y_mnist, labels)
        kmeans_results.append(
            {
                "n_clusters": n_clusters,
                "n_init": n_init,
                "ARI": ari,
                "NMI": nmi,
                "Silhouette": sil,
            }
        )

        print(
            f" -> ARI={ari:.4f}, NMI={nmi:.4f}, Silhouette={sil:.4f}"
        )

        if nmi > best_kmeans_score:
            best_kmeans_score = nmi
            best_kmeans_model = kmeans
            best_kmeans_config = {
                "n_clusters": n_clusters,
                "n_init": n_init,
            }

print("\nBest KMeans configuration (by NMI):", best_kmeans_config)
print("Best KMeans NMI:", best_kmeans_score)


In [ ]:
#%% 
#Results of K-Means in a table

import pandas as pd

kmeans_df = pd.DataFrame(kmeans_results)
print(kmeans_df.sort_values(by="NMI", ascending=False).reset_index(drop=True))


In [ ]:
#%% 
#GMM experiments with different hyperparameters with n_components and covariance_type

gmm_results = []
best_gmm_model = None
best_gmm_config = None
best_gmm_score = -np.inf  

n_components_list = [8, 10, 12]
covariance_types = ["full", "diag"]

for n_components in n_components_list:
    for cov_type in covariance_types:
        print(
            f"Training GMM: n_components={n_components}, "
            f"covariance_type={cov_type} ..."
        )
        gmm = GaussianMixture(
            n_components=n_components,
            covariance_type=cov_type,
            random_state=42,
        )
        gmm.fit(X_pca)
        labels = gmm.predict(X_pca)

        ari, nmi, sil = evaluate_clustering(X_pca, y_mnist, labels)
        gmm_results.append(
            {
                "n_components": n_components,
                "covariance_type": cov_type,
                "ARI": ari,
                "NMI": nmi,
                "Silhouette": sil,
            }
        )

        print(
            f" -> ARI={ari:.4f}, NMI={nmi:.4f}, Silhouette={sil:.4f}"
        )

        if nmi > best_gmm_score:
            best_gmm_score = nmi
            best_gmm_model = gmm
            best_gmm_config = {
                "n_components": n_components,
                "covariance_type": cov_type,
            }

print("\nBest GMM configuration (by NMI):", best_gmm_config)
print("Best GMM NMI:", best_gmm_score)


In [ ]:
#%% 
# Summarise GMM results in a simple table

gmm_df = pd.DataFrame(gmm_results)
print(gmm_df.sort_values(by="NMI", ascending=False).reset_index(drop=True))


In [ ]:
#%% 
# Compare best K-Means and best GMM based on NMI

print("Best KMeans (NMI):", best_kmeans_score, "config:", best_kmeans_config)
print("Best GMM   (NMI):", best_gmm_score, "config:", best_gmm_config)

if best_kmeans_score >= best_gmm_score:
    best_model_name = "KMeans"
    best_model = best_kmeans_model
    best_labels = best_kmeans_model.labels_
else:
    best_model_name = "GMM"
    best_model = best_gmm_model
    best_labels = best_gmm_model.predict(X_pca)

print("\nOverall best model:", best_model_name)


In [ ]:
#%% 
# Visualise clustering results for the best model:
# one row per cluster, five images per cluster

unique_clusters = np.unique(best_labels)
n_clusters_best = unique_clusters.shape[0]
samples_per_cluster = 5

fig, axes = plt.subplots(
    n_clusters_best,
    samples_per_cluster,
    figsize=(samples_per_cluster * 2, n_clusters_best * 2),
)

for k in range(n_clusters_best):
    cluster_idx = np.where(best_labels == k)[0]

    if cluster_idx.size == 0:
        # empty cluster (unlikely but possible); skip
        for j in range(samples_per_cluster):
            ax = axes[k, j]
            ax.axis("off")
        continue

    # if cluster has fewer than required samples, use replace=True
    replace = cluster_idx.size < samples_per_cluster
    chosen = rng.choice(
        cluster_idx,
        size=samples_per_cluster,
        replace=replace,
    )

    for j, i in enumerate(chosen):
        ax = axes[k, j]
        ax.imshow(X_mnist[i].reshape(img_size, img_size), cmap="gray")
        ax.axis("off")
        if j == 0:
            ax.set_ylabel(f"C{k}", rotation=0, labelpad=20, fontsize=10)

plt.suptitle(
    f"Best model: {best_model_name} – rows = clusters, columns = examples",
    y=1,
)
plt.tight_layout()
plt.show()
